# Trading News and Corporate Actions with BERT

<a href="" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Algo-trading on corporate actions by leveraging NLP. A replicationa and enhancement of the paper: *Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading (Zhou et al., Findings 2021)*.

We will perform the following steps:
1. Domain adaptation for financial articles by finetuning a BERT model with Masked Language Model (MLM) training on financial news and encyclopedia data. *Zhou et al.* utilized human annotators to label news articles with an event.
1. Bi-Level Event Detection: At Token-Level we detect events using a sequence labeling approach. At the higher Article-Level we will augment the corpus with 'CLS' token embedding which contains the the aggregate of all the article's embeddings, and concatenate it with the lower level tokens.
1. Recognize security Ticker, using string matching algorithm to recognize tickers within articles.
1. Create trading signals on the identified tickers.

```bibtex
@inproceedings{zhou-etal-2021-trade,
    title = "Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading",
    author = "Zhou, Zhihan  and
      Ma, Liqian  and
      Liu, Han",
    editor = "Zong, Chengqing  and
      Xia, Fei  and
      Li, Wenjie  and
      Navigli, Roberto",
    booktitle = "Findings of the Association for Computational Linguistics: ACL-IJCNLP 2021",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.findings-acl.186",
    doi = "10.18653/v1/2021.findings-acl.186",
    pages = "2114--2124",
}
```

# Notebook Environment

For a unified research environment, enable the flags below:

In [ ]:
UPGRADE_PY = False
INSTALL_DEPS = False
if INSTALL_DEPS:
  # %pip install -q tensorboard==2.15.2
  # %pip install -q tensorflow[and-cuda]==2.15.1
  # %pip install -q tensorflow==2.15.0
  # %pip install -q tensorflow-io-gcs-filesystem==0.36.0
  # %pip install -q tensorflow-text==2.15.0
  # %pip install -q tf_keras==2.15.1
  # %pip install -q tokenizers==0.15.2
  # %pip install -q torch==2.2.0+cpu
  # %pip install -q torch-xla==2.2.0+libtpu
  # %pip install -q torchdata==0.7.1
  %pip install -q transformers==4.38.2

if UPGRADE_PY:
  !mamba create -n py311 -y
  !source /opt/conda/bin/activate py312 && mamba install python=3.11 jupyter mamba -y

  !sudo rm /opt/conda/bin/python3
  !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3
  !sudo rm /opt/conda/bin/python3.10
  !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3.10
  !sudo rm /opt/conda/bin/python
  !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python

!python --version

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

# Transformers cannot use keras3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_USE_LEGACY_KERAS'] = '1'
IN_KAGGLE = IN_COLAB = False
!export CUDA_LAUNCH_BLOCKING=1
!export XLA_FLAGS=--xla_cpu_verbose=0

MODEL_PATH = "google-bert/bert-base-cased"
MODEL_PATH_TOKENIZER = "google-bert/bert-base-cased"
try:
  # https://www.tensorflow.org/install/pip#windows-wsl2
  import google.colab
  from google.colab import drive
  drive.mount('/content/drive')
  DATA_PATH = "/content/drive/MyDrive/EDT dataset"
  DATA_PATH = "/content/drive/MyDrive/investopediaBERT"
  IN_COLAB = True
  print('Colab!')
except:
  IN_COLAB = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ and not IN_COLAB:
    print('Running in Kaggle...')
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/uscorpactionnews"
    MODEL_PATH = "/kaggle/input/bert-base-cased-fine-tuned-on-investopedia/tensorflow2/investopedia/1"
    IN_KAGGLE = True
    print('Kaggle!')
elif not IN_COLAB and not IN_KAGGLE:
    IN_KAGGLE = False
    DATA_PATH = "./data/"
    print('Normal!')
    MODEL_PATH = "./models"


# Accelerators Configuration

If you have a GPU, TPU or in one of the collaborative notebooks. Configure your setup below:

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
import re
import pickle
from copy import deepcopy

from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import mixed_precision

print(f'Tensorflow version: [{tf.__version__}]')

tf.get_logger().setLevel('INFO')

#tf.config.set_soft_device_placement(True)
#tf.config.experimental.enable_op_determinism()
#tf.random.set_seed(1)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
except Exception as e:
  # Not an exception, just no TPUs available, GPU is fallback
  # https://www.tensorflow.org/guide/mixed_precision
  print(e)
  policy = mixed_precision.Policy('mixed_float16')
  mixed_precision.set_global_policy(policy)
  gpus = tf.config.experimental.list_physical_devices('GPU')
  if len(gpus) > 0:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, False)
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12288)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        strategy = tf.distribute.MirroredStrategy()

        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
    finally:
        print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
  else:
    # CPU is final fallback
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

def is_tpu_strategy(strategy):
    return isinstance(strategy, tf.distribute.TPUStrategy)

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

# Tokens, Sequences, and NER

Our corpus will be processed and labelled to 11 types of corporate events:
1. Acquisition(A)
1. Clinical Trial(CT)
1. Regular Dividend(RD)
1. Dividend Cut(DC)
1. Dividend Increase(DI)
1. Guidance Increase(GI)
1. New Contract(NC)
1. Reverse Stock Split(RSS)
1. Special Dividend(SD)
1. Stock Repurchase(SR)
1. Stock Split(SS).
1. No event (O)

Articles are structured as follows:

```json
'title': 'Title',
'text': 'Text Body',
'pub_time': 'Published datetime',
'labels': {
    'ticker': 'Security symbol',
    'start_time': 'First trade after article published',
    'start_price_open': 'The "Open" price at start_time',
    'start_price_close': 'The "Close" price at start_time',
    'end_price_nday': 'The "Close" price at the last minute of the following 1-3 trading day. If early than 4pm ET its the same day. Otherwise, it refers to the next trading day.',
    'end_time_1-3day': 'The time corresponds to end_price_1day',
    'highest_price_nday': 'The highest price in the following 1-3 trading',
    'highest_time_nday': 'The time corresponds to highest_price_1-3day',
    'lowest_price_nday': 'The lowest price in the following 1-3 trading day',
    'lowest_time_nday': 'The time corresponds to lowest_price_1-3day',
}
```

In [ ]:
NUM_LABELS = 12 # See Labels description above.
SPECIAL_TOKEN = 'CLS' # Use for classification and hidden state placeholder.
UNK_ID = -100 # Unknown token, ignored by loss
UNK = 'UNK'
OTHER_ID = 11
OTHER = 'O'

### Tokenizing News Text

The text body is tokenized, a simple example is shown below, including how the '[CLS]' (classification problem token) token is leveraged. BERT's transformer inputs expect of shape [batch_size, seq_length] the following inputs:
- "input_ids": token ids of the input sequences.
- "attention_mask": has value 1 at the position of all input tokens present before padding and value 0 for the padding tokens.
- "token_type_ids": the index of the input that created the input token. The first input segment (index 0) includes the start-of-sequence token and its end-of-segment token. The second segment (index 1, if present) includes its end-of-segment token. Padding tokens get index 0.

Transformers use self-attention mechanisms represent interactions amongst tokens and their contextual information in the input sequence as a weighted-sum. With this mechanism higher layers of the network will aggregate information from all other tokens in the sequence, in our case '[CLS]' will have such information.

Since we are passing the tokens as a batch, we need to give the tokenizer a maximum length on which to either PAD or TRUNCATE.

In [ ]:
from transformers import BertTokenizerFast, TFBertModel, BertConfig

# https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#berttokenizerfast
tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH_TOKENIZER)
model = TFBertModel.from_pretrained(MODEL_PATH)

text = ["When taken as a whole, the evidence suggests Cramer recommends “hot” stocks", "lending credence to the Hot Hand Fallacy in this context."]

tokenized_sequence = tokenizer.tokenize(text)
print(tokenized_sequence)

Note how words that weren't in the original model's vocabulary get split using '##', e.g. being *Cramer*'s name.

Running the full tokenizer will get the token IDs, their position (type ID) and the attention mask for BERT:

In [ ]:
MAX_LEN = 256 # Default 256, MAX 512
sample_inputs = inputs = tokenizer.encode_plus(
    text,
    add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
    max_length=MAX_LEN,  # Maximum length for padding/truncation, adjust as needed
    padding='max_length',
    return_tensors='tf',
    truncation=True
)
sample_inputs

Below is a textual representation of what the model will see, [CLS] showing the start of the clasification sequence, [SEP] to seperate sequences and [PAD]  to make the word embeddings the same size for batched predictions.

In [ ]:
tokenizer.decode(sample_inputs["input_ids"].numpy()[0])

The padding is signalled to be ignored by the model through the attention mask:

In [ ]:
sample_inputs["attention_mask"].shape

In [ ]:
sample_inputs["attention_mask"]

Since we are passing sequences, the model will need to know where one starts and another ends. This is signalled by sequence IDs.

Below we see that the first sequence is `0`, and the next is `1`, these are relevent in sequence classification, which we will be doing in this article:

In [ ]:
sample_inputs['token_type_ids']

If we run this from an out-of-the-box model forward pass, we get a sequence of tokens, though nothing is being predicted for now as there are no dense layers or a classification head yet - we only get the batched sequence as outputted by the last activation layer of the BERT model

The line `embedding = hidden_state[:, 0, :]` extracts the embeddings corresponding to the [CLS] token, which is often used as a representation of the entire sequence for classification tasks.

In [ ]:
outputs = model(sample_inputs['input_ids'])
hidden_state = outputs.last_hidden_state
embedding = hidden_state[:, 0, :]


# Get the hidden state with all info.

## Dataset

The paper authors provided a labelled dataset which we will process and prepare for the BERT inputs.

In our classification, we will hit some unknown tokens. The authors use ID -100 to denote this, we will use the pretrained tokenizer's ID:

In [ ]:
def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

train_ner_texts, train_ner_tags = read_wnut(os.path.join(DATA_PATH, 'Event_detection/train.txt'))
test_ner_texts, test_ner_tags = read_wnut(os.path.join(DATA_PATH, 'Event_detection/dev.txt'))

event_index = 0
for event_index, tags in enumerate(train_ner_tags):
    if any(tag != 'O' for tag in tags):
        break
print(f"event found at index: {event_index}")
print(*train_ner_texts[event_index])
print(*train_ner_tags[event_index])

In [ ]:
from transformers import BertTokenizerFast

def encode_tags(tags, encodings, tag2id, unk=UNK_ID):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in tqdm(zip(labels, encodings.offset_mapping), desc="encode_tags"):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset), dtype=int) * unk
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        max_len = len(doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)])
        doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)] = doc_labels[:max_len]
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

def encode_sequence_labels(ner_tags, tag2id, num_labels=MAX_LEN):
    seq_labels = []

    for tag in ner_tags:
        tag_set = set(tag)
        current_label = np.zeros([num_labels])
        if len(tag_set) == 1:
            current_label[tag2id[OTHER]] = 1
        else:
            # here is a bias, if a seq has another event, drop all others?
            # This is 'OHE' label
            tag_set.remove(OTHER)
            for tag in tag_set:
                current_label[tag2id[tag]] = 1
        seq_labels.append(list(current_label))

    return seq_labels

def load_and_cache_dataset(train_ner_texts, train_ner_tags,
                           test_ner_texts, test_ner_tags,
                           bert_model_tok=MODEL_PATH_TOKENIZER,
                           max_len=MAX_LEN,
                           num_labels=NUM_LABELS):
    tokenizer = BertTokenizerFast.from_pretrained(bert_model_tok)

    tags = deepcopy(train_ner_tags)
    tags.extend(test_ner_tags)
    unique_tags = list(set(tag for doc in tags for tag in doc))
    tag2id = {tag: id for id, tag in enumerate(sorted(unique_tags))}
    id2tag = {id: tag for tag, id in tag2id.items()}

    # Tokenize and encode labels for training and testing data
    train_encodings = tokenizer(train_ner_texts,
                                is_split_into_words=True,
                                return_offsets_mapping=True,
                                padding='max_length',
                                truncation=True,
                                max_length=max_len)
    train_ner_labels = encode_tags(train_ner_tags, train_encodings, tag2id, UNK_ID)
    train_seq_labels = encode_sequence_labels(train_ner_tags, tag2id, num_labels=num_labels)

    test_encodings = tokenizer(test_ner_texts,
                               is_split_into_words=True,
                               return_offsets_mapping=True,
                               padding='max_length',
                               truncation=True,
                               max_length=max_len)
    test_ner_labels = encode_tags(test_ner_tags, test_encodings, tag2id, UNK_ID)
    test_seq_labels = encode_sequence_labels(test_ner_tags, tag2id, num_labels=num_labels)

    # offset_mapping no longer needed
    train_encodings.pop("offset_mapping")
    test_encodings.pop("offset_mapping")

    return (train_encodings, train_ner_labels,
            test_encodings, test_ner_labels,
            train_seq_labels, test_seq_labels,
            tag2id, id2tag)

(train_encodings, train_ner_labels,
 test_encodings, test_ner_labels,
 train_seq_labels, test_seq_labels,
 tag2id, id2tag) = load_and_cache_dataset(train_ner_texts,
                                            train_ner_tags,
                                            test_ner_texts,
                                            test_ner_tags)
input_ids = np.array(test_encodings['input_ids'])
attention_mask = np.array(test_encodings['attention_mask'])
token_type_ids = np.array(test_encodings['token_type_ids']) if 'token_type_ids' in train_encodings else None
ner = np.array(test_ner_labels)
seq_label = np.array(test_seq_labels)
print("input_ids shape:", input_ids.shape)
print("attention_mask shape:", attention_mask.shape)
if token_type_ids is not None:
    print("token_type_ids shape:", token_type_ids.shape)

print("ner_labels shape:", ner.shape)
print("train_seq_labels shape:", seq_label.shape)

Let's check our tokenized dataset, we have the input ids being the word encodings and the entities labels:

In [ ]:
id2tag

In [ ]:
print(input_ids[:10].shape)
input_ids[:10]

In [ ]:
print(ner[:10].shape)
ner[:10]

In [ ]:
assert not np.isnan(input_ids).any()

## Inside, Outside, Beginning Tags

The IOB or BIO format is used in tagging chunks of words in NLP.

- I before a tag indicates that the tag is inside a chunk.
- O tag indicates that a token belongs to no chunk.
- B before a tag indicates that the tag is the beginning of a chunk that immediately follows another chunk without O tags between them.

## Imbalanced Dataset

Before you continue, looking at the tag frequencies - we see that our data set is imbalanced towards the `O` and `UNK` tags.



In [ ]:
from sklearn.utils.class_weight import compute_class_weight

unique, counts = np.unique(ner, return_counts=True)

# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
# https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
weights = compute_class_weight(class_weight="balanced", classes=np.unique(ner), y=ner.flatten())
weights_dict = {i: weights[i] for i in range(len(weights))}

df_tags = pd.DataFrame({'Tag ID': unique, 'Tag': (id2tag[id] if id in id2tag else UNK for id in unique),'Count': counts})
df_tags['Weight'] = df_tags['Tag ID'].map(lambda i: weights_dict.get(i, 0.))

df_weights = df_tags.sort_values(by='Tag ID', ascending=True)
df_weights.loc[df_weights['Tag ID'] == UNK_ID, 'Weight'] = 0. # Unkown tokens should be ignored totally.
df_weights.loc[df_weights['Tag ID'] == OTHER_ID, 'Weight'] = 0.05 # 'O' is over 75%! Need to reduce it within limits.

class_weights = df_weights[['Tag ID', 'Weight']].set_index('Tag ID').to_dict()['Weight']

df_weights

Give the class wieghts tot the model.

### Attention is all you Need

Vaswani et. al seminal paper *Attention is All You Need*, made self-attention and transformers mainstream.

Self-attention, calculates the relevance of each word in a sentence to every other word. This is done through queries (Q=XW^Q), keys (K=XW^K), and values (V=XW^V) transformed by a learned weight matrix (W) from the input embeddings (X). The attention score between two words is computed by taking the dot product of their queries and keys, followed by a softmax:

$ \text{A}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $

Where:
- Q represents the queries matrix of current items,
- K represents the keys matrix of items to compare against in the input sequence,
- V represents the values matrix, which are the dot product comparisons between Q and K,
- d_k represents the dimension of the keys and queries,
- A are the Attention wieghts.

In addition, the word embeddings will contain contextual information (dot poduct of A and V), represented as position added to the embedding. This plus the attention wieghts, captures dependencies and relationships.

## BERT Classifier

Built on top of a pretrained BERT (Bidirectional Encoder Representations from Transformers).BERT is an industry tested transformer-based model, pre-trained on a large corpus of text to generate contextual embeddings for input sequences.

We will use a small pre-trained cased base model with 12-layers + 768-hidden, 12-heads , and 110M parameters. This is the base model used in *Zhou et al. (2021)*. Later in the article, we will use larger BERT models that are more resource demanding to fine-tune. Each model has its own preprocesser, as text inputs need to be converted to token IDs.

The architecture can be summarized in 3 componets:
1. Input embeddings, attention masks and ID types for the preTrained BERT model. Bert applies transformer blocks with self-attention (attention captures language structures). The model outputs embedding sequences (last layer from BERT NxH) and a pooled summary derived from the first 'CLS' token(a 1XH vector).
1. The sequence outputs (NxH vector) is passed through dense layers and dropouts for the first NER classification, this maps the high-DIM outputs to logits. Padding of unknown tokens helps the model focus on the tasks.
1. NER logits are flattened and concatenated with the pooled summaries to form a new feature vector (NxH + H). The vector is passed again through dense and dropout layers to classify the event as one of the 11 identified (O is ignored).

# Model Build and Loss

In [ ]:
LEARN_RATE=5e-5 # 5e-5
LR_FACTOR=0.1
LR_MINDELTA=1e-4
EPOCHS=100
PATIENCE=10
BATCH_SIZE = 8 * strategy.num_replicas_in_sync # Default 8

In [47]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import AdamW, Adam
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.losses import Loss, SparseCategoricalCrossentropy, CategoricalFocalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import Metric, SparseCategoricalAccuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, Callback, ReduceLROnPlateau
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.mixed_precision import LossScaleOptimizer

from transformers import TFBertModel, BertConfig

class MultilabelBinaryAccuracy(Metric):
    def __init__(self, name='multilabel_binary_accuracy', labels_len=MAX_LEN, **kwargs):
        super(MultilabelBinaryAccuracy, self).__init__(name=name, **kwargs)
        self.correct_predictions = self.add_weight(name='correct', initializer='zeros')
        self.total_predictions = self.add_weight(name='total', initializer='zeros')
        self.labels_len = labels_len

    def update_state(self, y_true, y_pred, sample_weight=None):
        tf.print("MultilabelBinaryAccuracy")
        mask = tf.logical_and(tf.greater_equal(y_true, 0), tf.less(y_true, self.labels_len - 1))
        y_true_masked = tf.where(mask, y_true, tf.zeros_like(y_true))
        y_true_masked = tf.cast(y_true_masked, tf.float32)

        y_pred = tf.cast(tf.greater_equal(y_pred, 0.5), tf.float32)
        matches = tf.cast(tf.equal(y_true_masked, y_pred), tf.float32)
        matches *= tf.cast(mask, tf.float32)

        if sample_weight is not None:
            matches = tf.multiply(matches, tf.cast(sample_weight, tf.float32))

        self.correct_predictions.assign_add(tf.reduce_sum(tf.cast(matches, tf.float32)))
        self.total_predictions.assign_add(tf.cast(tf.size(matches, out_type=tf.int32), tf.float32))

    def result(self):
        tf.print("MultilabelBinaryAccuracy 1")
        return tf.cast(self.correct_predictions / self.total_predictions, tf.float32)

    def reset_states(self):
        self.correct_predictions.assign(0)
        self.total_predictions.assign(0)

class MaskedWeightedMultiClassBCE(Loss):
    def __init__(self, from_logits=False,
                 name='masked_weighted_multi_bce',
                 class_weight=None,
                 labels_len=MAX_LEN,
                 null_class=UNK_ID,
                 focal_gamma=None):
        super().__init__(name=name)
        self.from_logits = from_logits
        self.null_class = tf.cast(null_class, tf.float32)
        self.class_weight = None
        self.labels_len = labels_len
        if class_weight is not None:
            class_weights_list = [class_weight[i] for i in sorted(class_weight)]
            self.class_weight = tf.convert_to_tensor(class_weights_list, dtype=tf.dtypes.float32)
        self.focal_gamma = focal_gamma

    def call(self, y_true, y_pred):
        tf.print("MaskedWeightedMultiClassBCE")

        y_true = tf.cast(y_true, tf.float32)
        mask = tf.logical_and(tf.greater_equal(y_true, 0), tf.less(y_true, self.labels_len - 1))
        y_true_masked = tf.where(mask, y_true, tf.zeros_like(y_true))
        y_true_masked = tf.cast(y_true_masked, tf.float32)


        loss_fn = BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
        loss = loss_fn(y_true, y_pred)
        loss = loss * tf.cast(mask, tf.float32)
        return tf.reduce_mean(loss)

# https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalFocalCrossentropy
class MaskedWeightedSCCE(Loss):
    def __init__(self, from_logits=False,
                 name='masked_weighted_scce',
                 class_weight=None,
                 labels_len=MAX_LEN,
                 null_class=UNK_ID,
                 focal_gamma=None):
        super().__init__(name=name)
        self.from_logits = from_logits
        self.null_class = tf.cast(null_class, tf.float32)
        self.class_weight = None
        self.labels_len = labels_len
        if class_weight is not None:
            class_weights_list = [class_weight[i] for i in sorted(class_weight)]
            self.class_weight = tf.convert_to_tensor(class_weights_list, dtype=tf.dtypes.float32)
        self.focal_gamma = focal_gamma

    def call(self, y_true, y_pred):
        tf.print("MaskedWeightedSCCE")

        # Mask to exclude any class greater than null_class or invalid class entries to 0
        y_true = tf.cast(y_true, tf.float32)

        mask = tf.logical_and(tf.greater_equal(y_true, 0), tf.less(y_true, self.labels_len - 1))
        y_true_masked = tf.where(mask, y_true, tf.zeros_like(y_true))
        y_true_masked = tf.cast(y_true_masked, tf.float32)
        if tf.executing_eagerly():
            # tf.print(mask)
            # tf.print(y_true)
            tf.debugging.assert_greater(tf.reduce_sum(tf.cast(mask, tf.int32)),
                                        0, message="All data are masked!")

        # https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
        loss_fn = SparseCategoricalCrossentropy(from_logits=self.from_logits, reduction=tf.keras.losses.Reduction.NONE)

        if self.focal_gamma is not None:
            # inspired by: https://github.com/artemmavrin/focal-loss/blob/master/src/focal_loss/_categorical_focal_loss.py
            loss = loss_fn(y_true_masked, y_pred)
            y_pred = tf.clip_by_value(y_pred, clip_value_min=-100., clip_value_max=100.)
            proba = tf.nn.softmax(y_pred)
            y_true_rank = y_true_masked.shape.rank

            p_t = tf.gather(proba, tf.cast(y_true_masked, tf.int32),
                            axis=-1, batch_dims=y_true_rank)
            focal_modulation = tf.cast((1. - tf.clip_by_value(p_t, 0.01, 0.99)) ** self.focal_gamma, tf.float32)

            loss *= focal_modulation
            if self.class_weight is not None:
                loss *= tf.gather(self.class_weight, tf.cast(y_true_masked, tf.int32))
            if tf.executing_eagerly():
                # tf.print(p_t)
                # tf.print(focal_modulation)
                tf.debugging.assert_all_finite(focal_modulation, "Focal contains NaN or Inf")
        else:
          # We remove wieghts from focal loss as we zero the UNK class (ln(0)).
          loss = loss_fn(y_true_masked, y_pred,
                         sample_weight=tf.gather(self.class_weight,
                                                 tf.cast(y_true_masked, tf.int32)) if self.class_weight is not None
                                                 else None)
        loss = tf.cast(loss, tf.float32)
        loss *=  tf.cast(mask, tf.float32)

        # Avoid div by 0.
        sum_mask = tf.reduce_sum(tf.cast(mask, tf.float32))
        if tf.executing_eagerly():
            # tf.print(sum_mask)
            tf.debugging.assert_positive(sum_mask, message="sum_mask zeroed.")
        loss = (tf.reduce_sum(loss) / sum_mask
                      if sum_mask > 0.
                      else tf.constant(0., dtype=tf.float32))
        if tf.executing_eagerly():
            # tf.print(loss)
            tf.debugging.assert_positive(loss, message="Loss masked to zero.")

        return loss

# https://www.tensorflow.org/text/tutorials/bert_glue
def create_model(bert_model,
                 config,
                 num_labels=NUM_LABELS,
                 max_len=MAX_LEN,
                 unk=UNK_ID,
                 class_weight=None,
                 strategy=strategy):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name='token_type_ids')

    bert_outputs = bert_model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            return_dict=True)
    bert_sequence_output = tf.cast(bert_outputs.last_hidden_state, tf.float32)
    bert_pooled_output = tf.cast(bert_outputs.pooler_output, tf.float32)

    # Zero Logits that are paddings or special characters.
    mask = tf.cast(attention_mask, tf.float32 )
    mask = tf.expand_dims(mask, -1)
    masked_output = bert_sequence_output * mask

    ner_logits = Dropout(config.hidden_dropout_prob, name='Dropout_ner_1')(masked_output)
    ner_logits = Dense(2048, name='Dense_ner_1', kernel_initializer=GlorotUniform())(ner_logits)
    ner_logits = Dropout(config.hidden_dropout_prob, name='Dropout_ner_2')(ner_logits)
    ner_output = Dense(num_labels, name='ner_output', dtype='float32')(ner_logits)

    # combine NER predictions with entire sequence
    # NER shape is [batch_size, sequence_length, num_classes (12)].
    seq_input = tf.reshape(ner_output,
                           [tf.shape(ner_output)[0], tf.shape(ner_output)[1] * tf.shape(ner_output)[2]])
    seq_input = tf.concat([bert_pooled_output, seq_input], axis=1)

    seq_logits = Dropout(config.hidden_dropout_prob, name='Dropout_seq_1')(seq_input)
    seq_logits = Dense(2048, name='Dense_seq_1', kernel_initializer=GlorotUniform())(seq_logits)
    seq_logits = Dropout(config.hidden_dropout_prob, name='Dropout_seq_2')(seq_logits)
    # -1 as we don't classify 'O' Other - An article has 1+ events or None.
    seq_output = Dense(num_labels, name='seq_output', dtype='float32')(seq_logits)

    model = Model(inputs=[input_ids, attention_mask, token_type_ids],
                  outputs=[ner_output, seq_output])

    # https://www.tensorflow.org/api_docs/python/tf/keras/metrics/SparseCategoricalAccuracy
    optimizer = AdamW(learning_rate=LEARN_RATE)
    if not is_tpu_strategy(strategy):
      # TPUs already use bfloat16
      optimizer = LossScaleOptimizer(optimizer, dynamic=True)
    model.compile(optimizer=optimizer,
            loss={"ner_output": MaskedWeightedSCCE(from_logits=True, class_weight=None),
                  "seq_output": MaskedWeightedMultiClassBCE(from_logits=True, class_weight=None)},
            metrics={"ner_output": ['sparse_categorical_accuracy'],
                     "seq_output": [MultilabelBinaryAccuracy()]})

    return model

def get_tf_datasets(train_encodings, test_encodings, buffer_size=10000, batch_size=BATCH_SIZE):
    def create_dataset(encodings, ner_labels, seq_labels):
        input_ids = np.array(encodings['input_ids'])
        attention_mask = np.array(encodings['attention_mask'])
        token_type_ids = np.array(encodings['token_type_ids']) if 'token_type_ids' in train_encodings else None
        ner_labels = np.array(ner_labels)
        seq_labels = np.array(seq_labels)
        return tf.data.Dataset.from_tensor_slices((
            {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'token_type_ids': token_type_ids,
            },
            {
                'seq_output': seq_labels,
                'ner_output': ner_labels,
            },
        ))

    train_dataset = create_dataset(train_encodings, train_ner_labels, train_seq_labels)
    train_dataset = (train_dataset.shuffle(buffer_size=buffer_size)
                                    .batch(batch_size)
                                    .cache()
                                    .prefetch(tf.data.experimental.AUTOTUNE))
    test_dataset = create_dataset(test_encodings, test_ner_labels, test_seq_labels)
    test_dataset = (test_dataset.shuffle(buffer_size=buffer_size)
                                .batch(batch_size)
                                .cache()
                                .prefetch(tf.data.experimental.AUTOTUNE))

    return train_dataset, test_dataset

with strategy.scope():
    train_dataset, test_dataset = get_tf_datasets(train_encodings, test_encodings)

    config = BertConfig.from_pretrained(MODEL_PATH)
    config.num_labels = NUM_LABELS
    bert_model = TFBertModel.from_pretrained(MODEL_PATH, config=config)

    model = create_model(bert_model,
                         config,
                         num_labels=len(id2tag), class_weight=class_weights)
    # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
    tensorboard_callback = TensorBoard(log_dir='./logs',
                                        histogram_freq=2,
                                        embeddings_freq=2)
    # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    early_stopping = EarlyStopping(mode='min', patience=PATIENCE, start_from_epoch=1)
    lr_reducer = ReduceLROnPlateau(factor=LR_FACTOR,
                                    patience=0,
                                    min_delta=LR_MINDELTA,
                                    min_lr=LEARN_RATE/10.)
    #tf.debugging.enable_check_numerics() # - Assert if no Infs or NaNs go through. not for TPU!
    #tf.config.run_functions_eagerly(not is_tpu_strategy(strategy)) # - Easy debugging
    # https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
    history = model.fit(train_dataset,
                        epochs=EPOCHS,
                        callbacks=[tensorboard_callback, lr_reducer, early_stopping],
                        verbose="auto",
                        validation_data=test_dataset)
model.summary()

Some layers from the model checkpoint at ./models were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at ./models and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100
INFO:tensorflow:Error reported to Coordinator: in user code:

    File "/tmp/ipykernel_99332/2301138211.py", line 33, in update_state  *
        self.total_predictions.assign_add(tf.size(matches, out_type=tf.float32))

    TypeError: Value passed to parameter 'out_type' has DataType float32 not in list of allowed values: int32, int64
Traceback (most recent call last):
  File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tensorflow/python/training/coordinator.py", line 293, in stop_on_exception
    yield
  File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tensorflow/python/distribute/mirrored_run.py", line 387, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/__autograph_generated_file4g21w1rd.py", line 18, in run_step
    outputs = ag__.converted_call(ag__.ld(model).train_step, (ag__.ld(data),), None, fscope_1)
     

TypeError: in user code:

    File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1152, in train_step  *
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1246, in compute_metrics  *
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/adamd1985/anaconda3/envs/quant/lib/python3.11/site-packages/tf_keras/src/engine/compile_utils.py", line 620, in update_state  *
        metric_obj.update_state(y_t, y_p, sample_we

## NER Classifier

In [ ]:
traindata = train_dataset.unbatch().batch(1).take(1)

y1 = model.predict(traindata)
y1.shape

In [ ]:
predicted_classes = np.argmax(y1, axis=-1)

print("Logits (example):", y1)
print("Predicted classes:", predicted_classes)

In [ ]:
inputs, labels = next(iter(traindata))
labels

In [ ]:
labels.shape

In [ ]:
mask = tf.logical_and(tf.greater_equal(labels, 0), tf.less(labels, UNK_ID))
mask = tf.cast(mask, tf.float32)
losses = tf.keras.losses.sparse_categorical_crossentropy(labels, y1, from_logits=True, ignore_class=UNK_ID)
losses *= mask
mean_loss = tf.reduce_sum(losses) / tf.reduce_sum(tf.cast(mask, tf.float32))

print(f"Mask: {mask}")
print(f"Masked Losses: {losses.numpy()}")
print(f"Mean Loss: {mean_loss.numpy()}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_classification(history):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training vs. Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['sparse_categorical_accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_sparse_categorical_accuracy'], label='Validation Accuracy')
    plt.title('Training vs. Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_classification(history)

In [ ]:
from sklearn.metrics import f1_score, precision_recall_fscore_support, classification_report
import numpy as np

def print_classification_performanca(model, test_dataset):
    predictions = model.predict(test_dataset)
    predicted_label_indices = np.argmax(predictions, axis=-1)

    true_labels = np.array(test_ner_labels).flatten()
    predicted_labels = predicted_label_indices.flatten()

    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    print(classification_report(true_labels, predicted_labels, labels=range(0, UNK_ID), target_names=list(id2tag.values())))

    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
        true_labels, predicted_labels,
        average='weighted')

    ner_correct = np.sum(predicted_labels == true_labels)
    ner_total = len(true_labels)

    print('NER Accuracy: {:.2f}%'.format(100. * ner_correct / ner_total))
    print(f'Weighted Precision: {100. * weighted_precision:.2f}%, Recall: {100. * weighted_recall:.2f}%, F1: {100. * weighted_f1:.2f}%')

    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=list(id2tag.values()),
                yticklabels=list(id2tag.values()))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

print_classification_performanca(model, test_dataset)

The model has an obvious inbalance issue.

# Conclusion

TODO

## References

- [Zhou, Zhihan, Liqian Ma, and Han Liu. "Trade the event: Corporate events detection for news-based event-driven trading." arXiv preprint arXiv:2105.12825 (2021).](https://aclanthology.org/2021.findings-acl.186)
- [Hugging Face Transformers APIs](https://github.com/huggingface/transformers)
- [Hugging Face Model Repository and Spaces](https://huggingface.co/models)
- [Tensorflow Model Garden](https://github.com/tensorflow/models/tree/master/official/vision#table-of-contents)
- [Devlin, Jacob, et al. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).](https://arxiv.org/abs/1810.04805)
- [Google Pre-trained BERT Models.](https://github.com/google-research/bert?tab=readme-ov-file)
- [Ashish Vaswani and Noam Shazeer and Niki Parmar and Jakob Uszkoreit and Llion Jones and Aidan N. Gomez and Lukasz Kaiser and Illia Polosukhin, "Attention is All You Need". NIPS (2017)](https://research.google/pubs/attention-is-all-you-need/)
- [Lin, Tsung-Yi, et al. "Focal loss for dense object detection." Proceedings of the IEEE international conference on computer vision. 2017.](https://arxiv.org/pdf/1708.02002.pdf)
- [Universal Language Model Fine-tuning for Text Classification](https://arxiv.org/abs/1801.06146)

## Github

Article and code available on [Github](https://github.com/adamd1985/news-based-event-driven_algotrading)

Kaggle notebook available [here]()

Google Collab available [here]()

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.